In [ ]:
pip install idaes-pse

In [2]:
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

In [3]:
from idaes.dmf import DMF, magics
from pathlib import Path
idaes_home = Path("~/.idaes").expanduser()
if not idaes_home.exists():
    idaes_home.mkdir()
wspath = idaes_home / "workshop_workspace"
_dmf = DMF(wspath, create=True)

2022-07-12 03:51:10,409 [INFO] idaes.dmf.workspace: Using existing DMF workspace: /home/studio-lab-user/.idaes/workshop_workspace/config.yaml
2022-07-12 03:51:10,410 [INFO] idaes.dmf.workspace: Using existing path for new DMF workspace: /home/studio-lab-user/.idaes/workshop_workspace
2022-07-12 03:51:10,414 [INFO] idaes.dmf.dmfbase: Saving configuration location to: /home/studio-lab-user/.dmf


In [4]:
%dmf init ~/.idaes/workshop_workspace
%dmf list

2022-07-12 03:51:12,380 [INFO] idaes.dmf.dmfbase: Saving configuration location to: /home/studio-lab-user/.dmf


*Success!* Using workspace at "/home/studio-lab-user/.idaes/workshop_workspace"

| ID | Name(s) | Type | Modified | Description | 
| -- | ------- | ---- | -------- | ----------- |

True

In [5]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

In [6]:
from idaes.generic_models.properties.activity_coeff_models.BTX_activity_coeff_VLE \
    import BTXParameterBlock

m.fs.properties = BTXParameterBlock(
    default={
        "valid_phase":('Liq', 'Vap'),
        "activity_coeff_model": 'NRTL'
    }
)

In [7]:
# Select estimated NRTL parameter split to use
from ipywidgets import widgets
w = widgets.Dropdown(options=["1", "2"], value="1", description="Choose estimated parameter split",
                    style={"description_width": "initial"})
display(w)
use_split = w.value

Dropdown(description='Choose estimated parameter split', options=('1', '2'), style=DescriptionStyle(descriptio…

In [1]:
def NRTL_model(model, data):
    props = model.fs.properties
    # Fix NRTL specific variables
    # alpha values (set at 0.3)
    props.alpha["benzene", "benzene"].fix(0)
    props.alpha["benzene", "toluene"].fix(0.3)
    props.alpha["toluene", "toluene"].fix(0)
    props.alpha["toluene", "benzene"].fix(0.3)

    # initial tau values
    tau = data["parameters"]["tau"]
    props.tau["benzene", "benzene"].fix(0)
    props.tau["benzene", "toluene"].fix(tau["benzene,toluene"])
    props.tau["toluene", "toluene"].fix(0)
    props.tau["toluene", "benzene"].fix(tau["toluene,benzene"])

    # Set bounds on variables to be estimated
    props.tau["benzene", "toluene"].setlb(-5)
    props.tau["benzene", "toluene"].setub(5)

    props.tau["toluene", "benzene"].setlb(-5)
    props.tau["toluene", "benzene"].setub(5)

# Find & load NRTL parameters for the chosen split in the DMF
records = _dmf.find_one(name=f"BT NRTL est param{use_split}")
if records:
    # Create NRTL model with parameters
    NRTL_model(m, records.data)
else:
    print("No data found in DMF: Abort")
    exit(0)

NameError: name '_dmf' is not defined

In [ ]:
from idaes.generic_models.unit_models import Flash
m.fs.flash = Flash(default={"property_package": m.fs.properties})